In [55]:
#get area href
import requests
import re
import time
from bs4 import BeautifulSoup as bs

res = requests.get('http://www.fantasy-tours.com/Trip/Japan/Hokkaido.html?catalogNo=FB5A896DCA2BC274&areaNo=E7B0CCD39E3339B5')
res.encoding = 'utf-8'
soup = bs(res.text)

In [59]:
projectList = []
for ele in soup.select('#contentNav a'):
    fullDic = {}
    fullDic['agency'] = 'fantasy-tours'
    if re.search('E7B0CCD39E3339B5',ele['href']) != None:#E7B0CCD39E3339B5 is area nunber of japen
        fullDic['area'] = ele.text
        #get Area href info.
        resArea = requests.get('http://www.fantasy-tours.com'+ele['href'])
        resArea.encoding = 'utf-8'
        soupArea = bs(res.text)
        # pass if project href is not exist(IndexError)
        try:
            for eleArea in soupArea.select('.Ttext'):
                fullDic['title'] = eleArea.select('h3')[0].text
                #  flight href:  http://www.fantasy-tours.com/Product/travelList?prodNo={}
                fullDic['prodNo'] = re.search('prodNo=(.*)',eleArea.select('h3 a')[0]['href']).group(1)
                fullDic['href'] = 'http://www.fantasy-tours.com'+eleArea.select('.Tdate a')[0]['href']
                #get project href info.
                resInner = requests.get(fullDic['href'])
                resInner.encoding = 'utf-8'
                soupInner = bs(resInner.text)
                tourDic = {}
                #Tour info.
                i = 1
                for eleTour in soupInner.select('.tourDtitle'):
                    tourDic['day{}'.format(i)] = eleTour.text
                    i = i + 1
                tempTourDic = tourDic.copy()
                fullDic['tour'] = tempTourDic
                tempFullDic = fullDic.copy()
                projectList.append(tempFullDic)
        except IndexError:
            pass

In [60]:
import json
from json import load
from pprint import pprint
with open('fantasy_tours.json', 'w') as f:
     json.dump(projectList, f)